# Cluster analysis

In this notebook, we will look at how to debug blocked table similarities and the resulting clusters.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
# Assume that we have the "chartedIn" results
import takco

tables = takco.TableSet._load('../../output/chartedIn/1-reshape/*')

print(f"Got {sum(1 for _ in tables)} tables")
takco.preview( tables )

Got 549 tables


In [13]:
matchers = [
    {
        'name': 'headjacc',
        'class': 'CellJaccMatcher',
        'source': 'head',
        'tokenize': True,
        'cell_threshold': .99,
    },
    {
        'name': 'bodyjacc',
        'class': 'CellJaccMatcher',
        'source': 'body',
        'tokenize': True,
        'cell_threshold': 0.9
    }
]
tmp = '/tmp/'
clusters = takco.TableSet(tables).cluster(workdir=tmp, matchers=matchers, agg_func='mean', agg_threshold=.99)

print(f"Got {sum(1 for _ in clusters)} clusters")
takco.preview( clusters )

Clustering columns: 100%|██████████| 8/8 [00:00<00:00, 17.92it/s]


Got 8 clusters


In [29]:
import pandas as pd
pd.read_csv('/tmp/sims.csv')

,ti1,ti2,ci1,ci2,headjacc
0,0,0,0,0,1.0
1,0,0,0,1,1.0
2,0,0,1,0,1.0
3,0,0,1,1,1.0
4,1,1,2,2,1.0
...,...,...,...,...,...
259,32,32,87,87,1.0
260,32,32,88,88,1.0
261,32,32,89,89,1.0
262,32,32,90,90,1.0
